<a href="https://colab.research.google.com/github/Exter-dg/Scheduling-Algorithms/blob/master/OS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import plotly.figure_factory as ff

#Common function to generate gantt chart
#refer https://plotly.com/python/gantt/
#----------------------------------------------------------
                      #TODO
#Call chart() from all algos, and pass a list as describe below
#-----------------------------------------------------------

import random 

def chart(lst):#pass parameter that will have 2 list with process id,start,end (for rr id,start1,end1,id,start2,end2)
  df=[]
  for a in lst:
    df.append( dict(Task=a[0], Start=a[1] , Finish=a[2], Resource=a[0] ) )

  colors={}
  #colorlist=['red','yellow','blue','green','orange','brown','grey','pink']
  for a in lst:
    x=random.randint(0,250)
    y=random.randint(0,250)
    z=random.randint(0,250)
    col='rgb(%d,%d,%d)'%(x,y,z)
    colors.update({a[0]:col})
  
  fig = ff.create_gantt(df,colors=colors,group_tasks=True,index_col="Resource",height=400,width=1000)
  fig['layout']['xaxis'].update({'type': None}) #To remove datetime from x axis and display simple seconds
  fig.show()

chart( [["P1",0,2],["P2",2,4],["P1",4,6],["P3",7,9]] )

In [0]:
#1 FCFS
#2 Priority Scheduling
#3 Round-Robin

from tabulate import tabulate 


def turnAroundTime(size,bt,wt,tat):  
  for i in range(0,size):
    tat[i] = wt[i] + bt[i]


def fcfsWaitingTime(size,bt,at,wt):
  sumTime = [0] * size

  for i in range(1,size):
    sumTime[i] = sumTime[i-1] + bt[i-1]
    wt[i] = sumTime[i] - at[i]

    # when it arrives late and cpu is idle
    if(wt[i] < 0):
      wt[i] = 0


def fcfsAvgTime(size,bt,at):
  wt = [0] * size
  tat = [0] * size
  ct = [0] * size
  st=[0] * size

  fcfsWaitingTime(size,bt,at,wt)
  turnAroundTime(size,bt,wt,tat)

  totalWT=0
  totalTAT=0
  totalCT=0

  for i in range(0,size):
    totalTAT = totalTAT + tat[i]
    totalWT = totalWT + wt[i]
    ct[i] = tat[i] + at[i]
    totalCT = totalCT + ct[i]

  #print proper formatted table displaying all the info about fcfs
  headers=["PID","Arrival Time","Burst Time","Completion Time","Turnaround Time","Waiting Time"]
  info=[]
  for i in range(1,size+1):
    data=(i,at[i-1],bt[i-1],ct[i-1],tat[i-1],wt[i-1])
    info.append(data)
  print(tabulate(info,headers=headers))
  print("\n")
  
  avgWT = totalWT/size
  avgTAT = totalTAT/size
  avgCT = totalCT/size
  print("\nAverage completion time is %s seconds."%avgCT)
  print("Average waiting time is %s seconds."%avgWT)
  print("Average turnaround time is %s seconds."%avgTAT)
  print("\n\n")

  #Gantt chart
  #for fcfs start time = at + wt and bt+at+wt(finish time)
  lst = []

  for i in range(size):
    st[i]=at[i]+wt[i]
    s="P"+str(i+1)
    lst.append([s,st[i],bt[i]+wt[i]+at[i]])
    
  #print(lst)
  chart(lst)
  


In [0]:
#2 Priority Scheduling - Non preemtive and AT for all 0
# sort and then fcfs

def priorityScheduler(processes,size):
  processes = sorted(processes , key = lambda processes:processes[1] , reverse = True)
  at = [0] * size
  bt = [0] * size
  j=0
  for i in processes:
    bt[j] = i[1]
    j=j+1
  fcfsAvgTime(size,bt,at)


In [0]:
#SRTF
from collections import deque 

def srtfWaitingTime(processes, n, wt, ct, lst):  
   
    rt = [0] * n 
    for i in range(n):  
        rt[i] = processes[i][1] 
    complete = 0
    t = 0
    minm = 999999999
    short = 0
    check = False


    while (complete != n): 
          
        for j in range(n): 
            if ((processes[j][2] <= t) and (rt[j] < minm) and rt[j] > 0): 
                minm = rt[j] 
                short = j 
                check = True
        if (check == False): 
            t += 1
            continue
        rt[short] -= 1
        s="P"+str(processes[j][0]+1) #store id
        lst.append(s,t,t+1)


        minm = rt[short]  
        if (minm == 0):  
            minm = 999999999

        if (rt[short] == 0):  
            complete += 1
            check = False
            fint = t + 1
            ct[short] = fint
 
            wt[short] = (fint - proc[short][1] - proc[short][2]) 
  
            if (wt[short] < 0): 
                wt[short] = 0
             
        t += 1
   
def srtfTurnAroundTime(processes, n, wt, tat):  

    for i in range(n): 
        tat[i] = processes[i][1] + wt[i]  
  
 
def srtfAvgTime(processes, n):  
    wt = [0] * n 
    tat = [0] * n  
    ct = [0] * n
    lst =[]

    srtfWaitingTime(processes, n, wt, ct, lst)  
  
 
    srtfTurnAroundTime(processes, n, wt, tat)  
   
    
    headers=["PID","Arrival Time","Burst Time","Completion Time","Turnaround Time","Waiting Time"]
    info=[]
    for i in range(size):
      data=(processes[i][0],at[i][2],at[i][1],ct[i],tat[i],wt[i])
      info.append(data)
    
    print(tabulate(info,headers=headers))
    print("\n")

    totalWT = 0
    totatTAT = 0
    
    for i in range(n): 
      totalWT = totalWT + wt[i]  
      totalTAT = totalTAT + tat[i]  
    
    avgWT = totalWT/n
    avgTAT = totalTAT/n

    print("Average waiting time is %s seconds."%avgWT)
    print("Average turnaround time is %s seconds."%avgTAT)
    print("\n\n")
    chart(lst)

In [0]:
#3 Round Robin Scheduling
#at 2 5 6 6 
#bt 3 3 2 1
#q 2
#pop from at when 

#ready queue - > first check arrival time and put processes here 
#running queue-> one by one run then back to ready but first check for arrival

#need to store remaining bt too (store processes with id) 



import copy
from tabulate import tabulate


def rrCompletionTime(size,tempAt,ct,q,lst):

  #at has id,at,bt
  #(4, at = [[0,0,10],[1,1,4],[2,2,5],[3,3,3]], ,3)
 
  readyQueue = deque()
  runningQueue = deque()

  t = 0
  while(True):
    
    while( len(tempAt) != 0 and tempAt[0][1]<=t ):#process has arrived 
      readyQueue.append(tempAt.pop(0))

    if(len(readyQueue) == 0):
      t=t+1
      continue
      
    runningQueue.append(readyQueue.popleft())
    s="P"+str(runningQueue[0][0]+1) #id
    if( runningQueue[0][2] > q ):
      lst.append([s,t,t+q])
      runningQueue[0][2]-=q
      t+=q
      
      
    
      
      while( len(tempAt) != 0 and tempAt[0][1]<=t):
        readyQueue.append(tempAt.pop(0))
      if( runningQueue[0][2]==0 ): # when burst time left = quantum
        ct[runningQueue[0][0]]=t
        temp=runningQueue.popleft()
      else:
        readyQueue.append(runningQueue.popleft())

    else:
      lst.append([s,t,t+runningQueue[0][2]])
      t+=runningQueue[0][2]
      ct[runningQueue[0][0]]=t
      temp=runningQueue.popleft()
      #print(ct[runningQueue[0][0]])
      while( len(tempAt) != 0 and tempAt[0][1]<=t):
        readyQueue.append(at.pop(0))
    
    if(len(tempAt)==0 and len(readyQueue)==0):
      break;



def rrTurnAroundTime(size,at,tat,ct):
  for i in range(size):
    tat[i] = ct[i] - at[i][1]



def rrWaitingTime(size,wt,tat,at):
  for i in range(size):
    wt[i] = tat[i] - at[i][2];



def rrAvgTime(size,at,quantum):

  wt = [0] * size
  tat = [0] * size
  ct = [0] * size
  #tempAT=[0] * size
  lst=[]

  tempAt = copy.deepcopy(at)
 
  rrCompletionTime(size,tempAt,ct,quantum,lst)
  rrWaitingTime(size,wt,tat,at)
  rrTurnAroundTime(size,at,tat,ct)

  #print(at)
  #print(ct)
  headers=["PID","Arrival Time","Burst Time","Completion Time","Turnaround Time","Waiting Time"]
  info=[]
  for i in range(size):
    data=(at[i][0],at[i][1],at[i][2],ct[i],tat[i],wt[i])
    info.append(data)
  print("\n Quantum size is %s\n"%quantum)
  print(tabulate(info,headers=headers))
  print("\n")

  totalCT=0
  totalWT=0
  totalTAT=0

  for i in range(size):
    totalWT = totalWT + wt[i]
    totalTAT = totalTAT + tat[i]
    totalCT = totalCT + ct[i]

  avgWT = totalWT/size
  avgTAT = totalTAT/size
  avgCT = totalCT/size
  print("\nAverage completion time is %s seconds."%avgCT)
  print("Average waiting time is %s seconds."%avgWT)
  print("Average turnaround time is %s seconds."%avgTAT)
  print("\n\n")

  #Gantt chart
  chart(lst)

In [20]:

#main
#TODO main
#x='yes'
option = 0
while True:
  print("__________________________________________________________________\n") 
  print("CPU Scheduling Algorithms: " )
  print("__________________________________________________________________\n") 
  print("1. First Come First Serve") 
  print("2. Non-Preemtive Priority Scheduling") 
  print("3. Shortest Remaining Time First")
  print("4. Round Robin")
  print("5. Exit") 
  print( "-----------------------------------------------------------------") 
  print("Select > ",end=" ") 
  option = int(input())
  print("------------------------------------------------------------------\n") 

  if option is  1:
    print("FCFS SCHEDULING\n")
    print("-------------------")
    size = int(input("\nEnter no. of Processes : "))
    at=[0] * size
    bt=[0] * size
    for i in range(size):
      print("Enter Arrival Time for Process",i+1,end=' ')
      at[i]=int(input())
      print("")
    for i in range(size):
      print("Enter Burst Time for Process",i+1,end=' ')
      bt[i]=int(input())
      print("")
    
    fcfsAvgTime(size,at,bt)#size,bt,at
    print("\n\n")

  elif option is 2:
    print("PRIORITY SCHEDULING\n")
    print("-------------------")
    size = int(input("\nEnter no. of Processes : "))
    processes=[]
    for i in range(size):
      temp=[]
      print("Enter Priority for Process",i+1,end=' ')
      temp.append(int(input()))
      print("")
      print("Enter Burst Time for Process",i+1,end=' ')
      temp.append(int(input()))
      print("")
      processes.append(temp)
    priorityScheduler(processes,size) #[priosity,bt],size
    print("\n\n")

  elif option is 3:
    print("Shortest Remaining Time First")
    print("-------------------")
    size = int(input("\nEnter no. of Processes : "))
    processes=[]
    for i in range(size):
      temp=[]
      temp.append(i)
      print("Enter Burst Time for Process",i+1,end=' ')
      temp.append(int(input()))
      print("")
      print("Enter Arrival Time for Process",i+1,end=' ')
      temp.append(int(input()))
      print("")
      processes.append(temp)
    srtfAverageTime(processes,size)


  elif option is 4:
    print("ROUND ROBIN SCHEDULING\n")
    print("-------------------")
    size = int(input("\nEnter no. of Processes : "))
    processes=[]
    for i in range(size):
      temp=[]
      temp.append(i)
      print("Enter Arrival Time for Process",i+1,end=' ')
      temp.append(int(input()))
      print("")
      print("Enter Burst Time for Process",i+1,end=' ')
      temp.append(int(input()))
      print("")
      processes.append(temp)
    quantum=int(input("Enter the time quantum : "))
    #pass at list as [id,at,bt]
    rrAvgTime(size,processes,quantum) 
    print("\n\n")
    
  elif option is 5:
    break

  else:
    print("Wrong Input")

  #print("Coninue? yes/no")
  #x=input()


__________________________________________________________________

CPU Scheduling Algorithms: 
__________________________________________________________________

1. First Come First Serve
2. Non-Preemtive Priority Scheduling
3. Priority Scheduling
4. Round Robin
5. Exit
-----------------------------------------------------------------
Select >  3
------------------------------------------------------------------

PRIORITY SCHEDULING

-------------------

Enter no. of Processes : 5
Enter Arrival Time for Process 1 1

Enter Priority for Process 1 3

Enter Burst Time for Process 1 3

Enter Arrival Time for Process 2 2

Enter Priority for Process 2 4

Enter Burst Time for Process 2 5

Enter Arrival Time for Process 3 3
1

Enter Priority for Process 3 
Enter Burst Time for Process 3 1

Enter Arrival Time for Process 4 4

Enter Priority for Process 4 7

Enter Burst Time for Process 4 7

Enter Arrival Time for Process 5 5

Enter Priority for Process 5 8

Enter Burst Time for Process 5 4

  

__________________________________________________________________

CPU Scheduling Algorithms: 
__________________________________________________________________

1. First Come First Serve
2. Non-Preemtive Priority Scheduling
3. Priority Scheduling
4. Round Robin
5. Exit
-----------------------------------------------------------------
Select >  

KeyboardInterrupt: ignored